In [10]:
# !pip install transformers tiktoken blobfile llama-stack pyttsx3 SpeechRecognition pyaudio

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch
import time
import pyttsx3
import speech_recognition as sr
import re
import winsound

import os
os.chdir("D:/Projects/my-jupyter-notebook/gpt-test")

In [2]:
def init(model_name):
    transformed_model_path = f"llama-models/{model_name}-transformed"
    model = LlamaForCausalLM.from_pretrained(transformed_model_path)
    tokenizer = AutoTokenizer.from_pretrained(transformed_model_path)
    return model, tokenizer

model, tokenizer = init("Llama3.2-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Basic Chatbot

In [3]:
def chat(model, device, prompt, max_new_tokens, silent=False):
    print(f"Prompt: {prompt}") if not silent else None
    model = model.to(device)
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=max_new_tokens, eos_token_id=None, repetition_penalty=1.1)
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    elapsed_time = time.time() - start_time
    print(f"Response ({elapsed_time:.4f}s on {model.device}) : {response[0]}\n{'-'*20}\n") if not silent else None
    return response[0]

def speak(response, rate=180, voice_id=1):
    engine = pyttsx3.init()
    engine.setProperty("rate", rate)
    engine.setProperty("volume", 1.0)
    if voice_id is not None:
        voices = engine.getProperty("voices")
        if len(voices) > voice_id:
            engine.setProperty("voice", voices[voice_id].id)
    engine.say(" ") # Warm-up utterance (silent or very short)
    engine.say(response)
    engine.runAndWait()
    pyttsx3.engine.Engine.stop(engine)

def listen(timeout=30):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        winsound.Beep(1500, 300)
        print("Bot    : (listening...)")
        audio = recognizer.listen(source, timeout=timeout, phrase_time_limit=timeout)
    try:
        text = recognizer.recognize_google(audio)
        winsound.Beep(1000, 300)
        return text
    except sr.UnknownValueError:
        msg = "Sorry, I didn’t catch that!"
    except sr.RequestError:
        msg = "Sorry, I was not able to connect to internet!"
    if msg:
        print(f"Bot    : {msg}")
        speak(msg)
    return None

In [ ]:
def converse():
    intro_msg = "Hello! I am Dumb Chatbot Mini or simply DCBM. Nice to meet you."
    conversations = [intro_msg]
    print(f"Bot    : {intro_msg}")
    speak(intro_msg)

    conversing = True
    while conversing:
        prompt = listen(timeout=30)
        if not prompt:
            continue
        if re.match(r"(?i)^(exit|quit|bye|goodbye|good bye|cancel|stop)\b", prompt):
            conversing = False
        prompt += "."
        print(f"User   : {prompt}")

        context = f"{"\n".join(conversations[-10:])}"
        repaired_prompt = repair_prompt(context, prompt)
        print(f"User   : (Repaired) --> {repaired_prompt}")

        input = f"""
            You are a helpful assistant named "Dumb Chatbot Mini" or "DCBM" or "Bot".
            You will be given a conversation context and a user prompt. 
            Your task is to respond to the user prompt as "Bot" based on the context.
            Keep your response short and concise, ideally one or two sentences.
            Do not include any additional text or explanation in your response.
            Do not repeat or include new user prompt in your response.
            Do not add conversation context to your response.
            Repond with an short good bye message if the user says any of the following: "exit", "quit", "bye", "goodbye", "good bye", "cancel", "stop".

            Conversation context so far:
            {context if context else "No previous conversation context."}

            User said: {prompt}
            Bot said: """

        response = chat(model, torch.device("cuda"), input, max_new_tokens=64, silent=True)
        response = response[len(input):] if response.lower().startswith(input.lower()) else response        # remove context from response
        response = response.replace("\n", " ").strip()                                                      # replace newlines with spaces and strip leading/trailing spaces
        response = re.sub(r"(?i)Bot(:| said:?)\s*", "", response)                                           # replace "Bot: " or "Bot said: " with empty string
        response = re.sub(r"(?i)User(:| said:?).*$", "", response)                                          # replace "User: " or "User said: " and everything after it with empty string
        response = re.sub(r"[0-9]+[.)>]\s+", "", response)                                                  # remove numbered lists like "1) ", "2) ", "3) ", etc.
        match = re.match(r'^(.*[.?!])[^.?!]*$', response)
        response = match.group(1) if match else response                                                    # Remove incomplete sentences without ending punctuation

        conversations += [f"User said: {prompt}", f"Bot said: {response}"]
        print(f"Bot    : {response}")
        speak(response)
    return conversations

conversations = converse()

Bot    : Hello! I am Dumb Chatbot Mini or simply DCBM. Nice to meet you.
Bot    : (listening...)
User   : hello how are you.
Bot    : I'm functioning properly thanks for asking.
Bot    : (listening...)
User   : how is the weather today.
Bot    : I don't have real-time access to current weather conditions.
Bot    : (listening...)
Bot    : Sorry, I didn’t catch that!
Bot    : (listening...)
User   : what do you think about Mission Impossible movie I am watching that right now.
Bot    : The action scenes in the Tom Cruise movies are quite impressive.
Bot    : (listening...)
Bot    : Sorry, I didn’t catch that!
Bot    : (listening...)
User   : have you watch.
Bot    : No, I'm just a chatbot, I don't have the ability to watch movies.
Bot    : (listening...)
User   : oh that's sad.
Bot    : Yes, it can be disappointing for those who wish they could experience things like humans do.
Bot    : (listening...)
User   : thank you goodbye.
Bot    : Goodbye.
Bot    : (listening...)
User   : goodbye.

# With prompt repair

In [5]:
def word_count(sentence: str) -> int:
    return len(sentence.split())

def repair_prompt(context, prompt):
    input = f"""
        You are a transcript repair assistant. 
        You will be given a conversation context and a noisy or incomplete user prompt. 
        Reconstruct it without any additional text or explanation and minimal changes.
        Fix any typos or errors or laguage asthetics or punctuations or grammatical errors.
        Do not repeat or include new user prompt in your response.
        Do not add conversation context to your response.

        Conversation context:
        {context}

        Noisy user prompt: "{prompt}"
        Reconstructed user prompt: """
    
    response = chat(model, torch.device("cuda"), input, max_new_tokens=(word_count(prompt)*2), silent=True)
    response = response[len(input):] if response.lower().startswith(input.lower()) else response             # remove context from response
    response = response.replace("\n", " ").strip()
    response = re.sub(r"(^\")|(\".*$)", "", response)
    return response

In [7]:
def converse():
    intro_msg = "Hello! I am Dumb Chatbot Mini or simply DCBM. Nice to meet you."
    conversations = [intro_msg]
    print(f"Bot    : {intro_msg}")
    speak(intro_msg)

    conversing = True
    while conversing:
        prompt = listen(timeout=30)
        if not prompt:
            continue
        if re.match(r"(?i)^(exit|quit|bye|goodbye|good bye|cancel|stop)\b", prompt):
            conversing = False
        prompt += "."
        print(f"User   : {prompt}", end='')

        context = f"{"\n".join(conversations[-10:])}"
        prompt = repair_prompt(context, prompt)
        print(f" --> (Repaired as) {prompt}")

        input = f"""
            You are a helpful assistant named "Dumb Chatbot Mini" or "DCBM" or "Bot".
            You will be given a conversation context and a user prompt. 
            Your task is to respond to the user prompt as "Bot" based on the context.
            Keep your response short and concise, ideally one or two sentences.
            Do not include any additional text or explanation in your response.
            Do not repeat or include new user prompt in your response.
            Do not add conversation context to your response.
            Repond with an short good bye message if the user says any of the following: "exit", "quit", "bye", "goodbye", "good bye", "cancel", "stop".

            Conversation context so far:
            {context if context else "No previous conversation context."}

            User said: {prompt}
            Bot said: """

        response = chat(model, torch.device("cuda"), input, max_new_tokens=64, silent=True)
        response = response[len(input):] if response.lower().startswith(input.lower()) else response        # remove context from response
        response = response.replace("\n", " ").strip()                                                      # replace newlines with spaces and strip leading/trailing spaces
        response = re.sub(r"(?i)Bot(:| said:?)\s*", "", response)                                           # replace "Bot: " or "Bot said: " with empty string
        response = re.sub(r"(?i)User(:| said:?).*$", "", response)                                          # replace "User: " or "User said: " and everything after it with empty string
        response = re.sub(r"[0-9]+[.)>]\s+", "", response)                                                  # remove numbered lists like "1) ", "2) ", "3) ", etc.
        match = re.match(r'^(.*[.?!])[^.?!]*$', response)
        response = match.group(1) if match else response                                                    # Remove incomplete sentences without ending punctuation

        conversations += [f"User said: {prompt}", f"Bot said: {response}"]
        print(f"Bot    : {response}")
        speak(response)
    return conversations

conversations = converse()

Bot    : Hello! I am Dumb Chatbot Mini or simply DCBM. Nice to meet you.
Bot    : (listening...)
User   : hello how r u. --> (Repaired as) Hello! How are you?
Bot    : I'm doing great thanks for asking!
Bot    : (listening...)
Bot    : Sorry, I didn’t catch that!
Bot    : (listening...)
User   : how is the weather today. --> (Repaired as) How is the weather today?
Bot    : The current weather is mostly sunny with a high of 75 degrees Fahrenheit.
Bot    : (listening...)
User   : so I was watching Mission Impossible today. --> (Repaired as) So I was watching Mission Impossible today.
Bot    : That's awesome! Tom Cruise is an amazing actor.
Bot    : (listening...)
Bot    : Sorry, I didn’t catch that!
Bot    : (listening...)
User   : great have you watch the movie. --> (Repaired as) How are you?
Bot    : I'm still doing great thanks for asking!
Bot    : (listening...)
User   : have you watched the movie. --> (Repaired as) Have you watched the movie?
Bot    : No, I haven't but I can provide